In [1]:
# !pip install plotly kaleido datasets nbformat -U -q

In [2]:
import os

import datasets
import pandas as pd
from dotenv import load_dotenv
from huggingface_hub import login


load_dotenv(override=True)
login(os.getenv("HF_TOKEN"))

pd.set_option("max_colwidth", None)

OUTPUT_DIR = "output"

/Users/aymeric/venv/gaia/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
eval_ds = datasets.load_dataset("gaia-benchmark/GAIA", "2023_all")["validation"]
eval_ds = eval_ds.rename_columns({"Question": "question", "Final answer": "true_answer", "Level": "task"})
eval_df = pd.DataFrame(eval_ds)

In [4]:
pd.Series(eval_ds["task"]).value_counts()

2    86
1    53
3    26
Name: count, dtype: int64

# 1. Load all results

In [5]:
import glob


results = []
for f in glob.glob(f"{OUTPUT_DIR}/validation/*.jsonl"):
    df = pd.read_json(f, lines=True)
    df["agent_name"] = f.split("/")[-1].split(".")[0]
    results.append(df)

result_df = pd.concat(results)
result_df = result_df.drop(columns=["start_time", "end_time"])
result_df["prediction"] = result_df["prediction"].fillna("No prediction")

In [6]:
import re
from collections import Counter

from scripts.gaia_scorer import check_close_call, question_scorer


result_df["is_correct"] = result_df.apply(lambda x: question_scorer(x["prediction"], x["true_answer"]), axis=1)
result_df["is_near_correct"] = result_df.apply(
    lambda x: check_close_call(x["prediction"], x["true_answer"], x["is_correct"]),
    axis=1,
)

result_df["count_steps"] = result_df["intermediate_steps"].apply(len)


def find_attachment(question):
    matches = eval_df.loc[eval_df["question"].apply(lambda x: x in question), "file_name"]

    if len(matches) == 0:
        return "Not found"
    file_path = matches.values[0]

    if isinstance(file_path, str) and len(file_path) > 0:
        return file_path.split(".")[-1]
    else:
        return "None"


result_df["attachment_type"] = result_df["question"].apply(find_attachment)


def extract_tool_calls(code):
    regex = r"\b(\w+)\("
    function_calls = [el for el in re.findall(regex, code) if el.islower()]

    function_call_counter = Counter(function_calls)
    return function_call_counter


def sum_tool_calls(steps):
    total_count = Counter()
    for step in steps:
        if "llm_output" in step:
            total_count += extract_tool_calls(step["llm_output"])

    return total_count


# result_df["tool_calls"] = result_df["intermediate_steps"].apply(sum_tool_calls)

String  cannot be normalized to number str.
String 2017 Komo Mai Drive 900000 cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String 2 High fantasy A Song of Ice and Fire cannot be normalized to number str.
String  cannot be normalized to number str.
String 94 CFM for Cheater cannot be normalized to number str.
String  93 CFM for Cheater beater cannot be normalized to number str.
String 2017 Komo Mai Drive 900000 cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String No prediction cannot be normalized to number str

/Users/aymeric/Documents/Code/smolagents/examples/open_deep_research/scripts/gaia_scorer.py:52: UserWarning: Answer lists have different lengths, returning False.
  warnings.warn("Answer lists have different lengths, returning False.", UserWarning)


In [7]:
def get_thoughts(x):
    try:
        output = x[0]["task"]
        for y in x[1:]:
            try:
                if "observation" in y:
                    output += y["llm_output"] + "\nObservation:" + y["observation"]
                else:
                    output += y["llm_output"] + r"\Error:" + str(y["error"])
            except Exception:
                pass
        return output
    except Exception:
        return None


result_df["thoughts"] = result_df["intermediate_steps"].apply(lambda x: get_thoughts(x))

In [8]:
result_df["agent_name"].value_counts()

agent_name
code_gpt4o_03_february_text                         165
code_o1_03_february_ablation-toolcalling-manager    165
code_o1_01_february_text                            165
code_o3-mini_03_february_remove-navigational        165
code_o1_04_february_submission5                     165
code_o1_03_february_text_high-reasoning-effort      165
code_o1_03_february_remove-navigational             164
code_o1_03_february_fix-print-outputs               164
code_o1_04_february_submission                      162
code_o1_03_february_goodoldtext-unbroken            161
code_gpt4o_03_february_goodoldtext-unbroken         159
code_gpt4o_03_february_magenticbrowser              159
code_o1_03_february_fix-print-outputs2              156
code_gpt4o_03_february_magenticbrowser2             156
code_o1_04_february_submission-medium               125
code_o1_29-01_text                                  105
code_llama-3                                         90
code_o1_22-01_managedagent-summary_pl

# 2. Inspect specific runs

In [9]:
o1_vision = "code_o1_25-01_visioon"
o1_next = "code_o1_29-01_text"
o1 = "code_o1_01_february_text"

list_versions = [o1, o1_vision, o1_next]

# submission_selection_name = "react_code_llama3-70b_02-05_full-gaia-validation-code"
sel_df = result_df
# sel_df = sel_df.loc[
#     (result_df["agent_name"].isin(list_versions))
#     # & (~result_df["question"].isin(UNSOLVED_QUESTIONS))
# ]
sel_df = sel_df.reset_index(drop=True)
display(sel_df["agent_name"].value_counts())
sel_df = sel_df.drop_duplicates(subset=["agent_name", "question"])
display(sel_df.groupby("agent_name")[["task"]].value_counts())
print("Total length:", len(sel_df), "- is complete:", len(sel_df) == 165)
# assert sel_df["question"].value_counts().max() == len(list_versions), "Some questions are duplicate!"

agent_name
code_gpt4o_03_february_text                         165
code_o1_03_february_ablation-toolcalling-manager    165
code_o1_01_february_text                            165
code_o3-mini_03_february_remove-navigational        165
code_o1_04_february_submission5                     165
code_o1_03_february_text_high-reasoning-effort      165
code_o1_03_february_remove-navigational             164
code_o1_03_february_fix-print-outputs               164
code_o1_04_february_submission                      162
code_o1_03_february_goodoldtext-unbroken            161
code_gpt4o_03_february_goodoldtext-unbroken         159
code_gpt4o_03_february_magenticbrowser              159
code_o1_03_february_fix-print-outputs2              156
code_gpt4o_03_february_magenticbrowser2             156
code_o1_04_february_submission-medium               125
code_o1_29-01_text                                  105
code_llama-3                                         90
code_o1_22-01_managedagent-summary_pl

agent_name                                    task
code_gpt4o_03_february_goodoldtext-unbroken   2       84
                                              1       53
                                              3       22
code_gpt4o_03_february_magenticbrowser        2       83
                                              1       52
                                                      ..
code_o3-mini_03_february_remove-navigational  1       53
                                              3       26
code_qwen-coder-32B_03_february_text          2       22
                                              1       14
                                              3        7
Name: count, Length: 65, dtype: int64

Total length: 2809 - is complete: False


In [10]:
display("Average score:", sel_df.groupby("agent_name")[["is_correct"]].mean().round(3))
display(
    sel_df.groupby(["agent_name", "task"])[["is_correct", "is_near_correct", "count_steps", "question"]]
    .agg(
        {
            "is_correct": "mean",
            "is_near_correct": "mean",
            "count_steps": "mean",
            "question": "count",
        }
    )
    .rename(columns={"question": "count"})
)

'Average score:'

,is_correct
agent_name,
code_gpt4o_03_february_goodoldtext-unbroken,0.384
code_gpt4o_03_february_magenticbrowser,0.352
code_gpt4o_03_february_magenticbrowser2,0.365
code_gpt4o_03_february_text,0.376
code_llama-3,0.078
code_o1_01_february_text,0.491
code_o1_03_february_ablation-toolcalling-manager,0.327
code_o1_03_february_fix-print-outputs,0.518
code_o1_03_february_fix-print-outputs2,0.558


is_correct  \
agent_name                                   task               
code_gpt4o_03_february_goodoldtext-unbroken  1       0.452830   
                                             2       0.380952   
                                             3       0.227273   
code_gpt4o_03_february_magenticbrowser       1       0.480769   
                                             2       0.349398   
...                                                       ...   
code_o3-mini_03_february_remove-navigational 2       0.232558   
                                             3       0.153846   
code_qwen-coder-32B_03_february_text         1       0.357143   
                                             2       0.136364   
                                             3       0.142857   

                                                   is_near_correct  \
agent_name                                   task                    
code_gpt4o_03_february_goodoldtext-unbroken  1            0.452830   
                                             2            0.392857   
                                             3            0.227273   
code_gpt4o_03_february_magenticbrowser       1            0.480769   
                                             2            0.361446   
...                                                            ...   
code_o3-mini_03_february_remove-navigational 2            0.244186   
                                             3            0.153846   
code_qwen-coder-32B_03_february_text         1            0.357143   
                                             2            0.136364   
                                             3            0.142857   

                                                   count_steps  count  
agent_name                                   task                      
code_gpt4o_03_february_goodoldtext-unbroken  1        7.000000     53  
                                             2        8.511905     84  
                                             3       10.409091     22  
code_gpt4o_03_february_magenticbrowser       1        7.153846     52  
                                             2        8.168675     83  
...                                                        ...    ...  
code_o3-mini_03_february_remove-navigational 2        4.976744     86  
                                             3        6.615385     26  
code_qwen-coder-32B_03_february_text         1        5.428571     14  
                                             2        6.409091     22  
                                             3        6.571429      7  

[65 rows x 4 columns]

In [11]:
import plotly.express as px


cumulative_df = (
    (
        sel_df.groupby("agent_name")[["is_correct", "is_near_correct"]]
        .expanding(min_periods=1, axis=0, method="single")
        .agg({"is_correct": "mean", "is_near_correct": "count"})
        .reset_index()
    )
    .copy()
    .rename(columns={"is_near_correct": "index"})
)
cumulative_df["index"] = cumulative_df["index"].astype(int) - 1


def find_question(row):
    try:
        res = sel_df.loc[sel_df["agent_name"] == row["agent_name"], "question"].iloc[row["index"]][:50]
        return res
    except Exception:
        return ""


cumulative_df["question"] = cumulative_df.apply(find_question, axis=1)
# cumulative_df["question"] = [el[:50] for el in sel_df["question"].values]

# cumulative_df["is_correct"] = cumulative_df["is_correct"] * (165 - 68) / 165

px.line(
    cumulative_df,
    color="agent_name",
    x="index",
    y="is_correct",
    hover_data="question",
)

# 3. Dive deeper into one run

In [12]:
sel_df = result_df.loc[result_df["agent_name"] == o1]
print(len(sel_df))

165


### Count errors

In [13]:
import numpy as np


error_types = [
    "AgentParsingError",
    "AgentExecutionError",
    "AgentMaxIterationsError",
    "AgentGenerationError",
]
sel_df[error_types] = 0
sel_df["Count steps"] = np.nan


def count_errors(row):
    if isinstance(row["intermediate_steps"], list):
        row["Count steps"] = len(row["intermediate_steps"])
        for step in row["intermediate_steps"]:
            if isinstance(step, dict) and "error" in step:
                try:
                    row[str(step["error"]["error_type"])] += 1
                except Exception:
                    pass
    return row


sel_df = sel_df.apply(count_errors, axis=1)

/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/ipykernel_25011/2022001392.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/ipykernel_25011/2022001392.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/ipykernel_25011/2022001392.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [14]:
import plotly.express as px


aggregate_errors = (
    sel_df.groupby(["is_correct"])[error_types + ["Count steps"]].mean().reset_index().melt(id_vars=["is_correct"])
)

fig = px.bar(
    aggregate_errors,
    y="value",
    x="variable",
    color="is_correct",
    labels={
        "agent_name": "<b>Model</b>",
        "task": "<b>Level</b>",
        "aggregate_score": "<b>Performance</b>",
        "value": "<b>Average count</b>",
        "eval_score_GPT4": "<b>Score</b>",
    },
)
fig.update_layout(
    height=500,
    width=800,
    barmode="group",
    bargroupgap=0.0,
)
fig.update_traces(textposition="outside")
fig.write_image("aggregate_errors.png", scale=3)
fig.show()

### Inspect result by file extension type

In [15]:
display(
    result_df.groupby(["attachment_type"])[["is_correct", "count_steps", "question"]].agg(
        {"is_correct": "mean", "count_steps": "mean", "question": "count"}
    )
)

,is_correct,count_steps,question
attachment_type,,,
None,0.423799,4.959725,2185
csv,0.000000,7.750000,16
docx,0.571429,4.904762,21
jpg,0.142857,5.750000,28
jsonld,0.000000,6.600000,15
mp3,0.480000,4.500000,50
pdb,0.000000,4.444444,18
pdf,0.588235,4.137255,51
png,0.216783,4.412587,143


# 4. Ensembling methods

In [16]:
counts = result_df["agent_name"].value_counts()
long_series = result_df.loc[result_df["agent_name"].isin(counts[counts > 140].index)]

In [17]:
def majority_vote(df):
    df = df[(df["prediction"] != "Unable to determine") & (~df["prediction"].isna()) & (df["prediction"] != "None")]

    answer_modes = df.groupby("question")["prediction"].agg(lambda x: x.mode()[0]).reset_index()
    first_occurrences = (
        df.groupby(["question", "prediction"]).agg({"task": "first", "is_correct": "first"}).reset_index()
    )
    result = answer_modes.merge(first_occurrences, on=["question", "prediction"], how="left")

    return result


def oracle(df):
    def get_first_correct_or_first_wrong(group):
        correct_answers = group[group["is_correct"]]
        if len(correct_answers) > 0:
            return correct_answers.iloc[0]
        return group.iloc[0]

    result = df.groupby("question").apply(get_first_correct_or_first_wrong)

    return result.reset_index(drop=True)


display((long_series.groupby("agent_name")["is_correct"].mean() * 100).round(2))
print(f"Majority score: {majority_vote(long_series)['is_correct'].mean() * 100:.2f}")
print(f"Oracle score: {oracle(long_series)['is_correct'].mean() * 100:.2f}")

agent_name
code_gpt4o_03_february_goodoldtext-unbroken         38.36
code_gpt4o_03_february_magenticbrowser              35.22
code_gpt4o_03_february_magenticbrowser2             36.54
code_gpt4o_03_february_text                         37.58
code_o1_01_february_text                            49.09
code_o1_03_february_ablation-toolcalling-manager    32.73
code_o1_03_february_fix-print-outputs               51.83
code_o1_03_february_fix-print-outputs2              55.77
code_o1_03_february_goodoldtext-unbroken            53.42
code_o1_03_february_remove-navigational             53.66
code_o1_03_february_text_high-reasoning-effort      48.48
code_o1_04_february_submission                      49.38
code_o1_04_february_submission5                     55.15
code_o3-mini_03_february_remove-navigational        29.09
Name: is_correct, dtype: float64

Majority score: 58.18
Oracle score: 72.73


/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/ipykernel_25011/3287428472.py:20: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



### Submit

In [18]:
agent_run = "code_o1_04_february_submission5.jsonl"
df = pd.read_json(f"output/validation/{agent_run}", lines=True)
df = df[["task_id", "prediction", "intermediate_steps"]]
df = df.rename(columns={"prediction": "model_answer", "intermediate_steps": "reasoning_trace"})

In [19]:
df.to_json("submission.jsonl", orient="records", lines=True)